<a href="https://colab.research.google.com/github/ccg-esb/EvK/blob/main/py_pOXA48_SI5_communities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Computer experiments: multistrain communities

In [1]:
import numpy as np
import os
import csv
import sys
import matplotlib.pyplot as plt
import colorcet as cc
import time
import multiprocessing
import random
from tabulate import tabulate
import itertools


File structure

In [2]:
# Define your paths
root='/content/drive/MyDrive/SYNC_Projects/pOXA48/2023/code/py-files/'

path='./'
figPath = path+'figures/'
envPath = path+'env/'
dataPath = path+'data/'

strains_subsetE = [1, 2, 6, 9, 11, 15, 18, 20, 21, 24]
strains_subsetK = [25, 26, 29, 34, 37, 38, 39, 41, 43, 45]

strains_subset = strains_subsetE + strains_subsetK

print("subset: ",strains_subset)

strains = ['C001', 'C002',  'C006',  'C011',  'C012',  'C021',  'C022',  'C031',  'C051',  'C063',  'C094',  'C107',  'C115',  'C131',  'C141',  'C201',  'C227',  'C232',  'C247',  'C261',  'C286',  'C290',  'C302',  'C309',  'C324',  'K037',  'K038',  'K087',  'K094',  'K112',  'K114',  'K125',  'K141',  'K168',  'K177',  'K200',  'K201',  'K209',  'K213',  'K216',  'K224',  'K225',  'K241',  'K248',  'K249',  'K253',  'K257',  'K275',  'K285',  'K300']
plasmids = ['WT','TC']

tot_strains = int(len(strains))
cmap_strains = cc.glasbey_light[:tot_strains]


subset:  [1, 2, 6, 9, 11, 15, 18, 20, 21, 24, 25, 26, 29, 34, 37, 38, 39, 41, 43, 45]


In [3]:
from google.colab import drive
drive.mount('/content/drive')


os.chdir(root)
sys.path.insert(0, 'src/')
from pOXA48_S1 import *
from pOXA48_S2 import *
from pOXA48_S3 import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model parameters

In [4]:

B0 = 1e6 #Initial bacterial density
T = 24 #Duration of experimental season
S0 = 1 #Concentration of imiting resource
extinction_threshold=1 #Extinction threshold
alphas=[1e-10, 1e-12] #Antibiotic degradation rate
d=0.1 #Transfer dilution rate

A_max=65536*2 #Maximum antibiotic concentrations=[32768, 256, 1024, 32]

expe_params = {
    'B0': B0, #Initial bacterial density
    'A_max': A_max, #Maximum drug concentration
    'alphas': np.array(alphas), #Antibiotic degradation rate
    'T': T,  # Length of experiment
    'S0': S0,  # Resource concentration
    'd': d,  # Resource concentration
    'extinction_threshold': extinction_threshold,
}


## Load experimental parameters from files
(MCMC & Resistance & Conjugation permissiveness & MIC)

In [5]:
model_params = import_model_params("%smodel_params.csv"%(dataPath), expe_params)

display_model_params_stats(model_params)

╒═════════╤═════════════╤═══════════════════════╤════════════════════════════════════════════╤═════╕
│ Group   │ Parameter   │ Mean                  │ Range                                      │ N   │
╞═════════╪═════════════╪═══════════════════════╪════════════════════════════════════════════╪═════╡
│ E-TC    │ conj_rate   │ nan                   │ (nan, nan)                                 │ 25  │
├─────────┼─────────────┼───────────────────────┼────────────────────────────────────────────┼─────┤
│ E-TC    │ VKm         │ 6.3164e-10            │ (3.79e-10, 8.37e-10)                       │ 25  │
├─────────┼─────────────┼───────────────────────┼────────────────────────────────────────────┼─────┤
│ E-TC    │ rho         │ 842358780.8           │ (506376720.0, 1071282240.0)                │ 25  │
├─────────┼─────────────┼───────────────────────┼────────────────────────────────────────────┼─────┤
│ E-TC    │ seg_rate    │ 0.7843146083017115    │ (0.41282435738322215, 1.5263650048538724)

## Computer simulations: Serial transfers (Multiple Es)

In [6]:
def simulate_environment_multistrain(model_params, istrains, E, toPlot=False):
    start_time = time.time()  # Record the start time
    print('.', end="", flush=True)

    lbl_E = ''  # '%s_%s' % (str_E, indx_E)

    # Simulate transfers
    times_list, ys_list, strains_params_list = simulateTransfers_multistrain(model_params, istrains, E)

    # Get final points
    final_times, final_ys = get_final_points(times_list, ys_list)

    if toPlot:
        #print(E)
        #plot_environment(E, lbl_E)  # Plot environment
        print(strains_params_list)

        plotTransfers(times_list, ys_list, strains_params_list)  # Plot transfers
        plotTransfersFinalPoint(times_list, ys_list, strains_params_list)  # Plot final points

    # Analyze simulation results
    BpT, BfT, freqpT = analyze_simulation(final_ys)

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    return freqpT[-1]


def simulate_environments_multistrain(model_params, istrains, Es, toPlot=False):
    start_time = time.time()  # Record the start time
    freqpTs = []
    for E in Es:
        freqpT = simulate_environment_multistrain(model_params, istrains, E, toPlot)
        freqpTs.append(freqpT)

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    return freqpTs


In [7]:

num_days = 10
str_E = 'gaussian_noise'
iEs = [0, 1, 2, 4]
Es=load_environments(str_E, envPath, num_days, iEs)
print(Es)

[array([0.50621077, 0.22363581, 0.51050977, 0.5800726 , 0.67456203,
       0.20528631, 0.69927528, 0.29761472, 0.42275401, 0.57331148]), array([0.66053421, 0.2739223 , 0.6646922 , 0.64951632, 0.50267093,
       0.56632062, 0.55254091, 0.29834399, 0.58525063, 0.4599252 ]), array([0.20539574, 0.41635829, 0.32966776, 0.54121737, 0.3644348 ,
       0.60655987, 0.16206691, 0.40252758, 0.41533222, 0.46093486]), array([0.14294546, 0.68827552, 0.14610673, 0.56375391, 0.35210149,
       0.05310718, 0.52677606, 0.42660727, 0.58765231, 0.77313081])]


Here we run the simulations:

In [8]:
toPlot = False
max_strains = len(strains_subset)
istrains = [0, 2, 4] #example
print('istrains=', istrains)

freqpT = simulate_environments_multistrain(model_params, istrains, Es, toPlot)
print('\t\tfreqpT[-1]=%0.2f' % freqpT[-1])


istrains= [0, 2, 4]
.Total time elapsed: 5.89 seconds
.Total time elapsed: 10.57 seconds
.Total time elapsed: 10.39 seconds
.Total time elapsed: 3.69 seconds
Total time elapsed: 30.55 seconds
		freqpT[-1]=0.00


## Parallel computer simulations: Serial transfers (Multiple Es, Multiple strains)

In [9]:

def simulate_environments_parallel(model_params, istrains=[], Es=[], toPlot=False):
    start_time = time.time()  # Record the start time

    freqpTs = []  # Initialize the list to store the results

    # Create a multiprocessing Pool with the number of desired processes
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())  # Use all available CPU cores

    # Iterate over each E in Es
    for E in Es:
        # Iterate over each subset length
        #for subset_length in range(1, len(istrains) + 1, 5):  #tmp
        for subset_length in range(1, 6,2):  #tmp
            subset = istrains[:subset_length]
            print(f"\t{subset}: ", end="", flush=True)

            # Run the simulation for the current E and subset
            freqpT = simulate_environment_multistrain(model_params, subset, E, toPlot)
            freqpTs.append(freqpT)

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    return freqpTs




Here we run the simulation:

In [11]:

#istrains = [0, 2, 4]  # Example
#toPlot = True
#freqpTs = simulate_environments_parallel(model_params, istrains, Es, toPlot)


## Community combinatorics

In [12]:
def select_random_elements(strains_subset, num_elements):
    random_elements = random.sample(strains_subset, num_elements)
    return random_elements

In [13]:
toPlot = False
max_strains = 6 #len(strains_subset)
num_experiments = 10

freqpTs_experiments = []
for i in []: #range(num_experiments):
    istrains = select_random_elements(strains_subset, max_strains)
    print('istrains =', istrains)

    # Run the simulations in parallel

    freqpTs = simulate_environments_parallel(model_params, istrains, Es, toPlot)
    freqpTs_experiments.append(freqpTs)

#print(freqpTs_experiments)